# Solving Systems of Equations - Gauss Elimination

In this notebook we will begin learning how to solve systems of equations. That is
$$\begin{align*} 
7x_1 + 2x_2 - 3x_3 &= 12 \\
3x_1 - 8x_2 + 2x_3 &= 4 \\
-6x_1 + 2x_2 +9x_3 &= 9
\end{align*}$$

which we will write in matrix-vector form from now on as $\mathbf{A}\mathbf{x}=\mathbf{b}$

$$
\underbrace{\begin{bmatrix}
7 & 2 & -3 \\
3 & -8 & 2 \\
-6 & 2 & 9 \\
\end{bmatrix}}_{\mathbf{A}}
\underbrace{\begin{bmatrix}
x_1     \\
x_2     \\
x_3     \\
\end{bmatrix}}_{\mathbf{x}}
=
\underbrace{\begin{bmatrix}
12     \\
4     \\
9     \\
\end{bmatrix}}_{\mathbf{b}}
$$

The idea behind Gauss Elimination is to transform the system above ($\mathbf{A}\mathbf{x}=\mathbf{b}$) into an \emph{upper triangular} system ($\mathbf{A^*}\mathbf{x}=\mathbf{b^*}$) as seen below

$$
\underbrace{\begin{bmatrix}
A_{11} & A_{12} & A_{13} \\
0 & A^*_{22} & A^*_{23} \\
0 & 0 & A^*_{33} \\
\end{bmatrix}}_{\mathbf{A^*}}
\underbrace{\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}}_{\mathbf{x}}
=
\underbrace{\begin{bmatrix}
b_1 \\
b^*_2 \\
b^*_3 \\
\end{bmatrix}}_{\mathbf{b^*}}
$$

where any variable with an asterisk such as $\mathbf{A^*}$ indicates they have been modified from the original system.

Once you triangulate the system, you can easily solve for the last variable as
$$x_3 = A^*_{33} / b^*_3$$
Now that you have solved for $x_3$, you can move your attention to the next row up to solve for $x_2$. Notice how we are moving backwards. Expanding the second equation we get
$$A^*_{22}x_2 + A^*_{23}x_3 = b^*_2$$
and then re-arranging to solve for $x_2$ we get
$$x_2 = \frac{b^*_2 - A^*_{23}x_3}{A^*_{22}}$$
and we can repeat this same idea for each subsequent equation/row above until we find all of the $x_i$. This formula is generalized as
$$ x_i = \Bigg(b^*_i - \sum_{j=i+1}^{N} A^*_{ij}x_j \Bigg) \quad / \quad  A^*_{ii} $$



Since Gauss Elimination is a direct method (no iterating) the only source of error is round-off error. Of course, we always have round-off error if we are computing, but in iterative methods this round-off gets washed away with every iteration (correction to the solution). Since this does not happen with Gauss Elimination the round-off error accumulates all the way to the final solution. This is both the reason why for small systems Gauss is incredibly accurate (low number of FLOPs), but as the system increases in size round-off becomes significant and at a certain point it produces extremely large errors.

Gauss Elimination has algortihmic complexity of $O(n^3)$ so runtime of the code grows to be prohibitively long quickly. This reason, along with round-off error accumulation is why standard procedure is to use direct methods such as Gauss for small systems, but turn to iterative methods for large systems (n > 10k equations).

In [39]:
import numpy as np
A = np.array([[7.,2,-3],[3,-8,2],[-6,2,9]])
b = np.array([12., 4, 9])

In [40]:
def gauss(A, b):
    N = b.shape[0]
    x = np.zeros_like(b)
    s = np.zeros_like(b)

    for i in range(N - 1):
        s = -A[i + 1 : N, i] / A[i, i]
        A[i + 1 : N, :] += s[:, np.newaxis] * A[i, :]
        b[i + 1 : N] += s * b[i]

    x[-1] = b[-1] / A[-1, -1]

    for i in range(N - 2, -1, -1):
        x[i] = (b[i] - A[i, (i + 1) : N] @ x[(i + 1) : N]) / A[i, i]

    return x, A, b

Now let's run our test problem created at the beginning of the notebook. We will check our sanity with three things:
1. make sure $\mathbf{A}$ was triangulated correctly by printing it to the screen
2. check the residual vector $\lvert\mathbf{A}\mathbf{x}-\mathbf{b}\rvert$
3. check the solution against a proven solver from another library (i.e. the ```np.linalg.solve``` function)

In [41]:
# run the solver for our test problem
x, Atri, btri = gauss(A,b)
print("Triangulated A:\n", Atri)
print("Triangulated b:\n", btri)
# check the residual
print("Residual:\n", np.abs(A@x-b))
print("Absolute difference between our Gauss solver and np.linalg.solve()\n", x-np.linalg.solve(A,b))

Triangulated A:
 [[ 7.00000000e+00  2.00000000e+00 -3.00000000e+00]
 [ 0.00000000e+00 -8.85714286e+00  3.28571429e+00]
 [ 0.00000000e+00 -4.44089210e-16  7.80645161e+00]]
Triangulated b:
 [12.         -1.14285714 18.80645161]
Residual:
 [0. 0. 0.]
Absolute difference between our Gauss solver and np.linalg.solve()
 [0. 0. 0.]


It works!

## Where it breaks

But not always so great... there are caveats, of course.

The first thing to consider is the structure of $\mathbf{A}$. We can investigate if $\mathbf{A}$ is *diagonally dominant* which occurs when the magnitude of the diagonal element is greater than the sum of the magnitudes of the off-diagonal elements and this holds true for every row in the matrix. Even if one row is not diagonally dominant, the whole matrix is said to *not* be diagonally dominant. More precisely, $\mathbf{A}$ is diagonally dominant if
$$
\lvert A_{ii}\rvert > \sum_{j\neq i} \lvert A_{ij}\rvert \quad\text{for all } i $$
where $A_{ij}$ denotes the entry in the $i$ row and $j$ column.

Lets demonstrate why this is important. Suppose we wanted to solve a system as such

$$
\begin{bmatrix}
ϵ & 1 \\
1 & 1
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
=
\begin{bmatrix}
1 \\
2
\end{bmatrix}
$$

where $\epsilon$ is extremely small, around machine epsilon (~1e-16 for double precision). If that is the case, the solution should be

$$
\mathbf{x} ≈
\begin{bmatrix}
1 \\
1
\end{bmatrix}
$$

If we perform Gauss Elimination we get the triangulated system

$$
\begin{bmatrix}
ϵ & 1 \\
0 & 1-1/ϵ
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
=
\begin{bmatrix}
1 \\
2-1/ϵ
\end{bmatrix}
$$

and has the solution after backsubstituting

$$
\begin{gather*}
x_1 &= \frac{1-x_2}{ϵ} ≈ 0 \\
x_2 &= \frac{2 - 1/ϵ}{1-1/ϵ} ≈ 1
\end{gather*}
$$

which is wrong! Lets see this in action below

In [42]:
A = np.array([[1e-17, 1],[1, 1]])
b = np.array([1.,2])
x, Atri, btri = gauss(A,b)
print("x = \n", x)

x = 
 [0. 1.]


but if we use a number greater than machine epsilon such as 1e-15, we get

In [43]:
A = np.array([[1e-15, 1],[1, 1]])
b = np.array([1.,2])
x, Atri, btri = gauss(A,b)
print("x = \n", x)

x = 
 [0.99920072 1.        ]


Much better! Now, if all we do is simply switch the rows (pivot) we get the system

$$
\begin{bmatrix}
1 & 1 \\
ϵ & 1
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
=
\begin{bmatrix}
2 \\
1
\end{bmatrix}
$$

which is triangulated as

$$
\begin{bmatrix}
1 & 1 \\
0 & 1-ϵ
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
=
\begin{bmatrix}
2 \\
1-ϵ
\end{bmatrix}
$$

and has the solution after backsubstituting

$$
\begin{gather*}
x_1 &= 2 - x_2 ≈ 1 \\
x_2 &= \frac{1 - 1/ϵ}{1-1/ϵ} ≈ 1
\end{gather*}
$$



In [44]:

A = np.array([[1, 1],[1e-17, 1]])
b = np.array([2.,1])
x, Atri, btri = gauss(A,b)
print("x = \n", x)

x = 
 [1. 1.]


a perfect answer! the reason for this is because we have effectively triangulated the system by switching the rows. Another reason is we avoid diving by an extremely small number (the diagonal) when triangulating. 